In [3]:
!sudo apt install tesseract-ocr-khm
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-khm is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [5]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [4]:
import os
import random
import pathlib
import subprocess

import shutil

In [ ]:
khm_training_text_files = r'/content/drive/MyDrive/ABA_BANK/teseract_fine-tuning/langdata_lstm/khm/khm.training_text'
fontconf_dir = '/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/fonts.conf'
output_directory = r'/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesstrain/khm_data/khm-ground-truth'
training_text_files = r'/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/langdata/eng.training_text'

# Define the source and destination directories
source_directory = '/content/drive/MyDrive/ABA_BANK/teseract_fine-tuning/fonts'
destination_directory = '/usr/share/fonts/truetype/'

## Adding Font into collab environment path

In [ ]:
# Loop through all files in the source directory
for file in os.listdir(source_directory):
    # Check if the file is a .ttf file
    if file.endswith('.ttf'):
        # Construct full file paths
        source_file = os.path.join(source_directory, file)
        destination_file = os.path.join(destination_directory, file)

        # Copy the file
        shutil.copy(source_file, destination_file)
        print(f"Copied {file} to {destination_directory}")


Copied KhmerOS_muol.ttf to /usr/share/fonts/truetype/
Copied KhmerOS_muollight.ttf to /usr/share/fonts/truetype/
Copied KhmerOS_siemreap.ttf to /usr/share/fonts/truetype/
Copied KhmerOS_battambang.ttf to /usr/share/fonts/truetype/


In [ ]:
#update cache
!fc-cache -fv
!fc-list | grep "KhmerOS_muol"
!fc-list | grep "KhmerOS_muollight"
!fc-list | grep "KhmerOS_siemreap"
!fc-list | grep "KhmerOS_battambang"

/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 4 fonts, 2 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded
/usr/share/fonts/truetype/KhmerOS_muollight.ttf: Khmer OS Muol Light:style=Regular
/usr/share/fonts/truetype/KhmerOS_muol.ttf:

In [ ]:
lines = []

with open(khm_training_text_files, 'r', encoding='utf-8') as input_file:
    for line in input_file.readlines():
        lines.append(line.strip())

## Generating Images For A Font

In [ ]:
if not os.path.exists(output_directory):
    os.makedirs(output_directory)


random.shuffle(lines)
count = 10
lines = lines[:count]

line_count = 0
for line in lines:

    training_text_file_name = pathlib.Path(khm_training_text_files).stem
    line_training_text = os.path.join(output_directory, f'{khm_training_text_files}_{line_count}.gt.txt')

    # Write the text line to the ground truth text file
    with open(line_training_text, 'w') as output_file:
        output_file.write(line.strip() + '\n')

    file_base_name = f'khm_{line_count}'

    try:
        subprocess.run([
            'text2image',
            f'--fontconfig_tmpdir=(fontconf_dir)',
            '--font=Khmer OS Battambang',
            f'--text={line_training_text}',
            f'--outputbase={os.path.join(output_directory, file_base_name)}',
            '--max_pages=1',
            '--strip_unrenderable_words',
            '--leading=32',
            '--xsize=3600',
            '--ysize=480',
            '--char_spacing=1.0',
            '--exposure=0',
            '--unicharset_file=/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/langdata_lstm/khm/khm.unicharset'
        ], check=True)
    except subprocess.CalledProcessError as e:
        print(f'An error occurred while running text2image for line {line_count}: {e}')
    else:
        print(f'Generated image for line {line_count}')

    line_count += 1


Generated image for line 0
Generated image for line 1
Generated image for line 2
Generated image for line 3
Generated image for line 4
Generated image for line 5
Generated image for line 6
Generated image for line 7
Generated image for line 8
Generated image for line 9


## Generating Multiple Fonts

In [ ]:
# Assuming the following variables are defined: output_directory, lines, khm_training_text_files, fontconf_dir
unicharset_file = '/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/langdata_lstm/khm/khm.unicharset'
fonts = ['Khmer OS Battambang', 'Khmer OS Muol', 'Khmer OS Muol Light', 'Khmer OS Siemreap']

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

random.shuffle(lines)
count = 5000
lines = lines[:5000]

line_count = 0
for line in lines:
    training_text_file_name = pathlib.Path(khm_training_text_files).stem
    line_training_text = os.path.join(output_directory, f'{khm_training_text_files}_{line_count}.gt.txt')

    # Write the text line to the ground truth text file
    with open(line_training_text, 'w') as output_file:
        output_file.write(line.strip() + '\n')

    for font in fonts:
        file_base_name = f'khm_{line_count}_{font.replace(" ", "_")}'

        try:
            subprocess.run([
                'text2image',
                f'--fontconfig_tmpdir={fontconf_dir}',
                f'--font={font}',
                f'--text={line_training_text}',
                f'--outputbase={os.path.join(output_directory, file_base_name)}',
                '--max_pages=2000',
                '--strip_unrenderable_words',
                '--leading=32',
                '--xsize=3600',
                '--ysize=480',
                '--char_spacing=1.0',
                '--exposure=0',
                f'--unicharset_file={unicharset_file}'
            ], check=True)
        except subprocess.CalledProcessError as e:
            print(f'An error occurred while running text2image for line {line_count} with font {font}: {e}')
        else:
            print(f'Generated image for line {line_count} with font {font}')

    line_count += 1


Streaming output truncated to the last 5000 lines.
Generated image for line 3750 with font Khmer OS Battambang
Generated image for line 3750 with font Khmer OS Muol
Generated image for line 3750 with font Khmer OS Muol Light
Generated image for line 3750 with font Khmer OS Siemreap
Generated image for line 3751 with font Khmer OS Battambang
Generated image for line 3751 with font Khmer OS Muol
Generated image for line 3751 with font Khmer OS Muol Light
Generated image for line 3751 with font Khmer OS Siemreap
Generated image for line 3752 with font Khmer OS Battambang
Generated image for line 3752 with font Khmer OS Muol
Generated image for line 3752 with font Khmer OS Muol Light
Generated image for line 3752 with font Khmer OS Siemreap
Generated image for line 3753 with font Khmer OS Battambang
Generated image for line 3753 with font Khmer OS Muol
Generated image for line 3753 with font Khmer OS Muol Light
Generated image for line 3753 with font Khmer OS Siemreap
Generated image for l

## Fine-Tuning

In [10]:
!TESSDATA_PREFIX='/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesseract/tessdata' make training MODEL_NAME=Apex START_MODEL=eng TESSDATA='/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesseract/tessdata' MAX_ITERATIONS=10000


    You are using make version: 4.3
make: *** No rule to make target 'data/Apex-ground-truth/eng_10.lstmf', needed by 'data/Apex/all-lstmf'.  Stop.


In [ ]:
import os

tessdata_path = '/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesseract/tessdata'
os.environ['TESSDATA_PREFIX'] = tessdata_path


training_command = f"make training MODEL_NAME=Apex START_MODEL=eng TESSDATA={tessdata_path} MAX_ITERATIONS=10000"


!{training_command}

make: *** No rule to make target 'training'.  Stop.


In [7]:
%cd /content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesstrain

/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesstrain


In [8]:
!ls

data			       generate_wordstr_box.py	normalize.py	  requirements.txt
generate_gt_from_box.py        khm_data			ocrd-testset.zip  shuffle.py
generate_line_box.py	       LICENSE			plot		  src
generate_line_syllable_box.py  Makefile			README.md


In [9]:
!pwd

/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesstrain


In [ ]:
!make training MODEL_NAME=Apex START_MODEL=eng TESSDATA=../content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesseract/tessdata MAX_ITERATIONS=10000

    You are using make version: 4.3
make: *** No rule to make target 'data/Apex-ground-truth/eng_10.lstmf', needed by 'data/Apex/all-lstmf'.  Stop.


In [ ]:
TESSDATA_PREFIX=../tesseract/tessdata make training MODEL_NAME=Apex START_MODEL=eng TESSDATA=../tesseract/tessdata MAX_ITERATIONS=10